In [1]:
! pip install qiskit

     |████████████████████████████████| 8.5MB 3.7MB/s 
     |████████████████████████████████| 17.6MB 254kB/s 
     |████████████████████████████████| 204kB 43.9MB/s 
     |████████████████████████████████| 204kB 45.4MB/s 
     |████████████████████████████████| 2.1MB 20.9MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 1.0MB 32.0MB/s 
     |████████████████████████████████| 194kB 41.1MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 583kB 38.6MB/s 
     |████████████████████████████████| 3.2MB 27.7MB/s 
     |████████████████████████████████| 5.5MB 26.0MB/s 
  Created wheel for qiskit: filename=qiskit-0.24.0-cp37-none-any.whl size=2918 sha256=5061aa8dfb866c01544010088326efd5b2b4df653cd444952740d8fdec7a8d2e
  Stored in directory: /root/.cache/pip/wheels/22/fe/0a/26b7a409d1a9c3698b0319c7d009a97bffaae392d755158dca
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2

In [2]:
import matplotlib.pyplot as plt
import numpy as np

# Importing Qiskit
from qiskit import IBMQ, BasicAer, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

# Import basic plot tools
from qiskit.tools.visualization import plot_histogram

# Scipy
from scipy.optimize import rosen, differential_evolution

# Files
#from google.colab import files

In [11]:
def f1(t):
    return 1/2


def f2(x):
    return x ** 2 + x ** 3


def f3(x):
    return np.sin(x + 1)


def f4(x):
    return np.exp(x) + x


TEACHER = np.random.choice(np.linspace(-1, 1), 10)

In [4]:
def U_in(x, xq, qc):
    qc.rz(np.arccos(x ** 2), xq)
    qc.ry(np.arcsin(x), xq)


def U_out(i, j, xq, qc, theta, n):
    t = (i + j * n) * 3
    qc.rx(theta[t + 0], xq[i])
    qc.rz(theta[t + 1], xq[i])
    qc.rx(theta[t + 2], xq[i])


def Hamelton(n, qc):
    for i in range(n - 1):
        qc.cx(i, i + 1)
    #  for i in range(n - 1, 0, -1):
    #    qc.cx(i, i - 1)


def Bfunk(n, iter, count):
    b = [0] * n
    for k in count.keys():
        for i, l in enumerate(k):
            b[i] += int(l) * count[k]
    for i in range(n):
        b[i] /= iter
    return b


In [12]:
def Lfunk(Theta):
    s = np.pi /2
    Omega = [1/3, 1/3, 1/3]
    Iterations = 1000 
    deapth = 2   
    N = 3 
    L = 0 #np.dot(Omega, Omega) 
    for t in TEACHER:
        L += (f1(t) - fi(t, N, deapth, theta, Iterations, Omega)) ** 2
    return L


def fi(xi, N, deapth, theta, Iterations, Omega):

    xq = QuantumRegister(N, "x")
    yc = ClassicalRegister(1, "y")
    qc = QuantumCircuit(xq, yc)

    #qc.rz(theta[0], xq[0])
    qc.rx(theta[1], xq[0])

    qc.measure(xq[0], yc)
    job = execute(qc, backend, shots=Iterations)
    result = job.result()
    count = result.get_counts()
    
    return sum([float(k)*count[k] for k in count]) / Iterations


def GradLfunk(theta):
    gr = [0] * len(theta)
    s = np.pi /2
    Omega = [1/3, 1/3, 1/3]
    Iterations = 1000 
    deapth = 2   
    N = 3 
    for t in TEACHER:
        for i in range(len(theta)):
            theta[i] += s
            A = fi(t, N, deapth, theta, Iterations, Omega)
            theta[i] -= 2 * s
            B = fi(t, N, deapth, theta, Iterations, Omega)
            theta[i] += s
            C = (A - B)
            if i == 0:
              print(C)
            gr[i] += (fi(t, N, deapth, theta, Iterations, Omega) - f1(t)) * C / np.sin(s)
            #print(np.array(gr))
    return np.array(gr)

In [7]:
def ftestgrad(x):
    return np.array([2 * (x[0] - 1), 2 * (x[1] + 1)])


In [13]:
backend = BasicAer.get_backend('qasm_simulator')
theta = np.array([0, 1], dtype=np.float64)

etta = .01
for i in range(10):
    grad = GradLfunk(theta)
    theta -= etta * GradLfunk(theta)
    #print(grad)
    #print(Lfunk(theta))


0.022999999999999993
0.0010000000000000009
0.035
0.03
-0.014999999999999986
0.0020000000000000018
0.006000000000000005
-0.010000000000000009
0.0
-0.010999999999999982
0.013999999999999985
0.02099999999999999
0.007000000000000006
-0.015000000000000013
-0.031000000000000028
-0.006000000000000005
0.015999999999999986
0.0050000000000000044
-0.0030000000000000027
-0.0020000000000000018
0.013000000000000012
0.007000000000000006
0.0020000000000000018
0.0010000000000000009
-0.01999999999999999
0.0
-0.010000000000000009
-0.020000000000000018
-0.0050000000000000044
0.038000000000000006
-0.03
0.006000000000000005
-0.00899999999999998
0.0020000000000000018
0.01100000000000001
0.0020000000000000018
0.006000000000000005
0.024999999999999994
0.014999999999999986
-0.028999999999999998
0.0050000000000000044
-0.0030000000000000027
-0.0030000000000000027
-0.027000000000000024
0.034999999999999976
-0.009000000000000008
0.029000000000000026
-0.01899999999999999
0.02100000000000002
0.0
0.01200000000000001
-